In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/data')

Mounted at /content/drive


In [2]:
import numpy as np
import csv
import pandas as pd

df = pd.read_csv("12cars_data.csv",header=0)

#clean mile
mile = []
for i in df['เลขไมล์']:
    if i.split('-')[0]=='':
        m = 0
    elif len(i.split('-'))==1:
        m = int(i)
    elif i.split('-')[1][-1]=='k' or i.split('-')[1][-1]=='K':
        m = (int(i.split('-')[0])+int(i.split('-')[1][:-1]))*1000/2
    mile.append(m)
df['mile'] = mile

#concat ยี่ห้อ+รุ่น
# data["Name"]= data["Name"].str.cat(new, sep =", ") 
df['brand'] = df['ยี่ห้อ'].str.cat(df['รุ่น'],sep =",")
# item_counts = df["col1"].value_counts(normalize=True)
# df['brand'].value_counts(normalize=True)

#create old
df['ปี']
old = []
for i in df['ปี']:
    old.append(2020-int(i))
df['old'] = old

#clean engine
engine = []
count = 0
for i in df['ขนาดเครื่องยนต์']:
    try:
        if int(i) < 500 or int(i) > 10000:
            engine.append(0)
            count += 1
        else:
            engine.append(int(i))
    except:
        engine.append(0)
df['engine'] = engine

#create gear
df['gear'] = df['ระบบเกียร์']

#create seat
df['seat'] = df['จำนวนที่นั่ง'].replace('-',0)
# df['seat'].value_counts(normalize=False)

#price


df['colour'] = df['สี']

df['fuel'] = df['ประเภทเชื้อเพลิง']
# df.info()

# df['date'].isna().sum()
from datetime import date
post = []
for i in df['date']:
    try:
        d0 = date(int(i.split('-')[2]), int(i.split('-')[1]), int(i.split('-')[0]))
    except:
        d0 = date(2563, 11, 12)
        
    d1 = date(2563, 11, 30)
    delta = d1 - d0
#     post.append(int(delta.days/30))  #post-month 
    post.append(int(delta.days/90))  #post-month 
df['post'] = post


data = df[['price','brand','mile','old','engine','gear','seat','province','colour','fuel','post']]

data.isnull().sum()
data = data.dropna()

data = data[data.engine >= 10]
data.describe()

data.to_csv('car_clean.csv',index=False)

In [3]:
#make model

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

df = pd.read_csv("car_clean.csv",header=0)

x = df.drop(['price'],axis=1)
y = df['price']
y = np.ravel(y);

scaler = MinMaxScaler(feature_range=(0,1))
# data["price"]=scaler.fit_transform(data[["price"]])
x["mile"]=scaler.fit_transform(x[["mile"]])
x["old"]=scaler.fit_transform(x[["old"]])
x["engine"]=scaler.fit_transform(x[["engine"]])
x["post"]=scaler.fit_transform(x[["post"]])

x = pd.get_dummies(x, dummy_na = False, columns=['brand', 'gear', 'seat','province','colour','fuel'])

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

nEstimators = 500
clf = RandomForestRegressor(n_estimators=nEstimators, max_features="sqrt")
clf = clf.fit(x_train, y_train)

y_pred = clf.predict(x_test);

scores = cross_val_score(clf,x_test,y_test, cv = 5)
mean_abs_error = sum(abs(y_test-y_pred))/len(y_test)
mean_per_error = mean_abs_error/np.mean(y_test)

print("Scores:",scores)
print("Mean absolute error:",mean_abs_error)
print("Mean percent error: ",mean_per_error)

Scores: [0.83186599 0.67450644 0.78844768 0.63497573 0.71708908]
Mean absolute error: 119812.61146369687
Mean percent error:  0.15722953128867523


In [4]:
x

,mile,old,engine,post,"brand_Alfa,156","brand_Alfa,GT","brand_Alfa,GTV","brand_Alfa,Spider","brand_Aston,DB11","brand_Aston,Rapide","brand_Aston,V8","brand_Audi,A1","brand_Audi,A3","brand_Audi,A4","brand_Audi,A5","brand_Audi,A6","brand_Audi,A7","brand_Audi,A8","brand_Audi,Q2","brand_Audi,Q3","brand_Audi,Q5","brand_Audi,Q7","brand_Audi,Q8","brand_Audi,R8","brand_Audi,RS3","brand_Audi,RS5","brand_Audi,S5","brand_Audi,TT","brand_Audi,TTS","brand_Austin,A90","brand_BMW,116i","brand_BMW,118i","brand_BMW,218Ci","brand_BMW,218i","brand_BMW,220i","brand_BMW,316i","brand_BMW,318Ci","brand_BMW,318i","brand_BMW,320Ci","brand_BMW,320d",...,seat_2,seat_3,seat_4,seat_5,seat_6,seat_7,seat_8,seat_11,seat_12,seat_15,seat_16,seat_18,province_coupé,province_กรุงเทพและปริมณฑล,province_ภาคกลาง,province_ภาคตะวันตก,province_ภาคตะวันออก,province_ภาคอีสาน,province_ภาคเหนือ,province_ภาคใต้,colour_ขาว,colour_ครีม,colour_ชมพู,colour_ดำ,colour_ทอง,colour_น้ำตาล,colour_น้ำเงิน,colour_ฟ้า,colour_ม่วง,colour_ส้ม,colour_อื่นๆ,colour_เขียว,colour_เงิน,colour_เทา,colour_เหลือง,colour_แดง,fuel_Diesel,fuel_Hybrid,fuel_Natural,fuel_Petrol
0,0.000118,0.029851,0.085277,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,0.000255,0.074627,0.288777,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0.000516,0.194030,0.180170,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0.000081,0.044776,0.132409,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,0.000304,0.119403,0.321248,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27653,0.000210,0.119403,0.132566,0.666667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
27654,0.000139,0.089552,0.132566,0.666667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
27655,0.000019,0.194030,0.132409,0.933333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
27656,0.000354,0.238806,0.211538,0.933333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [5]:
#save model
import pickle

filename = 'x_model.sav'
pickle.dump(clf, open(filename, 'wb')) #wb-write in binary mode

# filename = 'car_model.pha'
# pickle.dump(clf, open(filename, 'wb')) #wb-write in binary mode

In [6]:
#predict

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import pickle

#load model
# filename = 'carprice_model.pha'
filename = 'x_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
    
def predictCar(brand,province,colour,mile,old):
#     brand = 'Toyota,Corolla'
#     province = 'กรุงเทพและปริมณฑล'
#     colour = 'ขาว'
#     mile = 147500.0
#     old = 8
    def find_basic_data_car(brand,province,colour,mile,old):
    
        def findMax(v):
        #     v = 'engine'
            max_name = ''
            max_n = 0
            for i in df[v].unique():
                if df[df[v]==i].shape[0] > max_n:
                    max_n = df[df[v]==i].shape[0]
                    max_name = i
            return max_name

        df = pd.read_csv("car_clean.csv",header=0)
        df = df[df.brand == brand]
        df = df[df.old == 8]

        engine = findMax('engine')
        gear = findMax('gear')
        seat = findMax('seat')
        fuel = findMax('fuel')

        return engine,gear,seat,fuel




    engine,gear,seat,fuel = find_basic_data_car(brand,province,colour,mile,old)
    post = 0

    df = pd.read_csv("car_clean.csv",header=0)

    x = df.drop(['price'],axis=1)
    y = df['price']
    y = np.ravel(y);

    # t = ['Toyota,Corolla',147500.0,8,1986,'A',5,'กรุงเทพและปริมณฑล','ขาว','Petrol',1] #5567
    t = [brand,mile,old,engine,gear,seat,province,colour,fuel,post]
    h = ['brand','mile','old','engine','gear','seat','province','colour','fuel','post']
    df2 = pd.DataFrame([t], columns=h)
    x2 = x.append(df2, ignore_index=True)



    scaler = MinMaxScaler(feature_range=(0,1))

    x2["mile"]=scaler.fit_transform(x2[["mile"]])
    x2["old"]=scaler.fit_transform(x2[["old"]])
    x2["engine"]=scaler.fit_transform(x2[["engine"]])
    x2["post"]=scaler.fit_transform(x2[["post"]])

    x2 = pd.get_dummies(x2, dummy_na = False, columns=['brand', 'gear', 'seat','province','colour','fuel'])

    y_pred2 = loaded_model.predict([x2.loc[x2.shape[0]-1]])

    return y_pred2[0]



brand = 'Toyota,Corolla'
province = 'กรุงเทพและปริมณฑล'
colour = 'ขาว'
mile = 147500.0
old = 8
p = predictCar(brand,province,colour,mile,old)
p

297905.986

In [7]:
import pandas as pd
import csv
df = pd.read_csv("car_clean.csv",header=0)
df

,price,brand,mile,old,engine,gear,seat,province,colour,fuel,post
0,389000.0,"Toyota,Yaris",47500.0,2,1197,A,5,กรุงเทพและปริมณฑล,แดง,Petrol,0
1,429000.0,"Nissan,Teana",102500.0,5,2488,A,5,กรุงเทพและปริมณฑล,ขาว,Petrol,0
2,239000.0,"Honda,Civic",207500.0,13,1799,A,5,กรุงเทพและปริมณฑล,ดำ,Petrol,0
3,449000.0,"Toyota,Vios",32500.0,3,1496,A,5,กรุงเทพและปริมณฑล,เทา,Petrol,0
4,289000.0,"Toyota,Hilux",122500.0,8,2694,M,7,กรุงเทพและปริมณฑล,ขาว,Petrol,0
...,...,...,...,...,...,...,...,...,...,...,...
27653,465000.0,"Honda,Jazz",84634.0,8,1497,A,5,ภาคตะวันตก,ขาว,Petrol,10
27654,559000.0,"Honda,Jazz",56002.0,6,1497,A,5,ภาคตะวันตก,ขาว,Petrol,10
27655,259000.0,"Toyota,Yaris",7500.0,13,1496,A,5,กรุงเทพและปริมณฑล,แดง,Petrol,14
27656,450000.0,"Toyota,Rav4",142500.0,16,1998,A,5,กรุงเทพและปริมณฑล,ดำ,Petrol,14


In [8]:
df.colour.unique()

array(['แดง', 'ขาว', 'ดำ', 'เทา', 'น้ำตาล', 'ส้ม', 'อื่นๆ', 'เงิน',
       'น้ำเงิน', 'เหลือง', 'ฟ้า', 'ทอง', 'เขียว', 'ครีม', 'ชมพู', 'ม่วง'],
      dtype=object)